<a href="https://colab.research.google.com/github/sashera1/CS-4770-Homeworks/blob/main/CS_4770_NLP_Fall_2025_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment 2: N-gram Language Models

### *CS 4770 Natural Language Processing (Fall 2025)*

In this assignment, you will implement N-gram language models (Unigram, Bigram, and Trigram) using Python. You will train your models on a given corpus, apply smoothing techniques, and evaluate your models using perplexity on a test set.

### Tasks
1. **Python implementation of N-gram Language Models**: Train Unigram, Bigram, and Trigram models.
2. **Smoothing Techniques**: Implement add-k smoothing and interpolation.
3. **Language Model Evaluation**: Using n-gram models to generate texts and calculate perplexity to evaluate your models.

### Note
Remember to delete ```raise NotImplementedError``` before completing the TODOs.


### 0. Setup

Run this cell to setup necessary packages and environments. No need to modify anything in this cell.

In [1]:
# Import required packages
import numpy as np
import torch
import nltk
from sklearn.model_selection import train_test_split

# Set random seeds
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Download the Reuters corpus
nltk.download('brown')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

### 1. Python implementation of N-gram Language Models

Import the previously downloaded corpus, split it into training and test sets, and apply necessary preprocessing. No need to modify anything in this cell.


In [2]:
from nltk.corpus import brown, reuters, inaugural

# Load the Reuters corpus and split into training and test sets
corpus = brown.sents()[:2000]

# Add a start and end token for each sentence
corpus = [ ['[BOS]'] + [w.lower() for w in sent] + ['[EOS]'] for sent in corpus]

train_corpus, test_corpus = train_test_split(corpus, test_size=0.2, random_state=42)

#### 1.1 Unigram Language Model
Calculate unigram probabilities based on the training set by completing the TODO.

In [3]:
def train_unigram_lm(corpus):
    unigram_counts = {}
    unigram_model = {}

    for sentence in corpus:
        for word in sentence:
            if word not in unigram_counts:
                unigram_counts[word] = 1
            else:
                unigram_counts[word] += 1

    total_words = sum(unigram_counts.values())

    # TODO: Calculate unigram probabilities using unigram counts
    for word in unigram_counts:
      unigram_model[word] = unigram_counts[word]/total_words

    return unigram_counts, unigram_model

unigram_counts, unigram_model = train_unigram_lm(train_corpus)

#### 1.2 Bigram Language Model
Calculate bigram probabilities based on the training set by completing the TODO.


In [5]:
def train_bigram_lm(corpus):
    unigram_counts, _ = train_unigram_lm(corpus)
    bigram_counts = {}
    bigram_model = {}

    for sentence in corpus:
        for i in range(len(sentence) - 1):
            bigram = (sentence[i], sentence[i + 1])
            if bigram not in bigram_counts:
                bigram_counts[bigram] = 1
            else:
                bigram_counts[bigram] += 1

    # TODO: Calculate bigram probabilities using bigram counts
    for bigram in bigram_counts:
      bigram_model[bigram] = bigram_counts[bigram]/unigram_counts[bigram[0]]

    return bigram_counts, bigram_model

bigram_counts, bigram_model = train_bigram_lm(train_corpus)

#### 1.3 Trigram Language Model
Calculate trigram probabilities based on the training set by completing the TODO.

In [6]:
def train_trigram_lm(corpus):
    bigram_counts, _ = train_bigram_lm(corpus)
    trigram_counts = {}
    trigram_model = {}

    for sentence in corpus:
        for i in range(len(sentence) - 2):
            trigram = (sentence[i], sentence[i + 1], sentence[i + 2])
            if trigram not in trigram_counts:
                trigram_counts[trigram] = 1
            else:
                trigram_counts[trigram] += 1

    # TODO: Compute trigram probabilities using trigram counts
    for trigram in trigram_counts:
      trigram_model[trigram] = trigram_counts[trigram]/bigram_counts[(trigram[0],trigram[1])]

    return trigram_counts, trigram_model

trigram_counts, trigram_model = train_trigram_lm(train_corpus)

#### 1.4 Using N-gram Language Models

Just run this cell and copy the printed output to your latex. No need to modify anything in this cell.

In [7]:
# Calculate the probability of a sample sentence using the Unigram, Bigram, and Trigram models

def calulate_unigram_prob(sentence, unigram_model):
    for idx in range(len(sentence)):
        if idx == 0:
            prob = 1 # assume the sentence always starts with [BOS]
        else:
            prob *= unigram_model.get(sentence[idx], 0)
    return prob


def calculate_bigram_prob(sentence, bigram_model):
    for idx in range(len(sentence)):
        if idx == 0:
            prob = 1 # assume the sentence always starts with [BOS]
        else:
            prob *= bigram_model.get((sentence[idx-1], sentence[idx]), 0)
    return prob

def calculate_trigram_prob(sentence, trigram_model, bigram_model):
    for idx in range(len(sentence)):
        if idx == 0:
            prob = 1 # assume the sentence always starts with [BOS]
        elif idx == 1:
            prob *= bigram_model.get((sentence[0], sentence[1]), 0)
        else:
            prob *= trigram_model.get((sentence[idx-2], sentence[idx-1], sentence[idx]), 0)
    return prob

sample_sentence = train_corpus[0]
print(f'Sample sentence: {sample_sentence}')

# Test the Unigram, Bigram, and Trigram models
unigram_prob = calulate_unigram_prob(sample_sentence, unigram_model)
bigram_prob = calculate_bigram_prob(sample_sentence, bigram_model)
trigram_prob = calculate_trigram_prob(sample_sentence, trigram_model, bigram_model)

print(f"Unigram probability: {unigram_prob}")
print(f"Bigram probability: {bigram_prob}")
print(f"Trigram probability: {trigram_prob}")


Sample sentence: ['[BOS]', 'five', 'candidates', 'seek', 'the', 'place', 'vacated', 'by', 'secretary', 'hugh', 'g.', 'stout', '.', '[EOS]']
Unigram probability: 4.122943312604837e-42
Bigram probability: 1.3991150205742658e-14
Trigram probability: 0.00015624999999999998


### 2. Python Implementation of Smoothing

Implement add-k smoothing and interpolation for your N-gram models by completing the TODO. Compare the differences in probabilities before and after applying smoothing.

In [11]:
def add_k_smoothing(unigram_counts, bigram_counts, k):
    smoothed_bigram_model = {}

    for word_1 in unigram_counts:
        for word_2 in unigram_counts:
            #will be called like this: .get(word before, word, k) let word_1 be word_before, word_2 be word
            #p(word|word before) > p(word_2, word_1)
            # #(word before, word)+k / #(word before)+k|V| > #(word_1,word_2)+k / #(word_1)+k|V|
            if (word_1,word_2) in bigram_counts:
              numerator_sequence = bigram_counts[(word_1,word_2)]
            else:
              numerator_sequence = 0
            numerator = numerator_sequence + k
            denominator = unigram_counts[word_1] + k*len(unigram_counts)
            smoothed_probability = numerator/denominator
            smoothed_bigram_model[(word_1,word_2)] = smoothed_probability

    return smoothed_bigram_model

smoothed_bigram_model = add_k_smoothing(unigram_counts, bigram_counts, 0.4)

sample_sentence = test_corpus[5]

print(f'sample_sentence: {sample_sentence}')
print(f'Bigram probabilities (before smoothing): {calculate_bigram_prob(sample_sentence, bigram_model)}')
print(f"Smoothed Bigram probabilities (add-k): {calculate_bigram_prob(sample_sentence, smoothed_bigram_model)}")

sample_sentence: ['[BOS]', "you'll", 'probably', 'get', 'a', 'ball', 'bat', 'on', 'the', 'head', '.', '[EOS]']
Bigram probabilities (before smoothing): 0.0
Smoothed Bigram probabilities (add-k): 1.6706506325434276e-35


In [12]:
def interpolate_trigram_prob(sentence, trigram_model, bigram_model, unigram_model, lambdas):
    for idx in range(len(sentence)):
        if idx == 0:
            prob = 1 # assume the sentence always starts with [BOS]
        elif idx == 1:
            prob *= bigram_model.get((sentence[0], sentence[1]), 0)
        else:
            # TODO: Compute interpolated trigram probabilities
            # prob *= p_interpolate(x_i | x_i-2, x_i-2)
            # = l_1*p(x_i) + l_2*p(x_i | x_i-1) + 1_3*p(x_i | x_i-1,x_i-2)


            unigram_term = lambdas[0]*unigram_model[sentence[idx]]
            bigram_term = lambdas[1]*bigram_model.get((sentence[idx-1],sentence[idx]),0.0)
            trigram_term = lambdas[2]*trigram_model.get((sentence[idx-2],sentence[idx-1],sentence[idx]),0.0)
            p_interpolate = unigram_term + bigram_term + trigram_term
            prob *= p_interpolate

    return prob

sample_sentence = test_corpus[8]

print(f'Trigram probabilities (before interpolation): {calculate_trigram_prob(sample_sentence, trigram_model, bigram_model)}')
print(f'Interpolated Trigram probabilities: {interpolate_trigram_prob(sample_sentence, trigram_model, bigram_model, unigram_model, [0.1, 0.3, 0.6])}')

Trigram probabilities (before interpolation): 0.0
Interpolated Trigram probabilities: 3.0825212308551035e-08



### 3. N-gram Language Model Evaluation

Evaluate your trained N-gram models with perplexity on the test set by completing the TODO. Answers to both TODOs are the same.

In [13]:
def calculate_bigram_perplexity(sentence, smoothed_bigram_model):
    """
    Calculates the perplexity of a sentence using the add-k smoothed bigram model.
    """
    perplexity = 0
    for idx in range(len(sentence)):
        if idx == 0:
            prob = 1  # assume the sentence always starts with [BOS]
        else:
            prob = smoothed_bigram_model.get((sentence[idx - 1], sentence[idx]), 1e-10)  # use a small prob value for unseen bigrams to avoid nan perplexity

        perplexity += np.log(prob)

    # TODO: Compute perplexity
    perplexity *= (-1/len(sentence))
    perplexity = np.exp(perplexity)

    return perplexity

# Calculate PPL for add-k smoothed bigram model
bigram_perplexity = []
for sentence in test_corpus:
    perplexity = calculate_bigram_perplexity(sentence, smoothed_bigram_model)
    bigram_perplexity.append(perplexity)

print(f"Smoothed Bigram Perplexity (add-k): {np.mean(bigram_perplexity)}")

Smoothed Bigram Perplexity (add-k): 466002.42391135567


In [14]:
def calculate_trigram_perplexity(sentence, trigram_model, bigram_model, unigram_model, lambdas):
    """
    Calculates the perplexity of a sentence using the interpolated trigram model.
    """
    perplexity = 0
    for idx in range(len(sentence)):
        if idx == 0:
            prob = 1  # assume the sentence always starts with [BOS]
        elif idx == 1:
            prob = bigram_model.get((sentence[0], sentence[1]), 1e-10)  # use a small prob value for unseen bigrams to avoid nan perplexity
        else:
            prob = lambdas[0] * unigram_model.get(sentence[idx], 1e-10) + lambdas[1] * bigram_model.get((sentence[idx-1], sentence[idx]), 1e-10) + lambdas[2] * trigram_model.get((sentence[idx-2], sentence[idx-1], sentence[idx]), 1e-10)

        perplexity += np.log(prob)

    # TODO: Compute perplexity
    perplexity *= (-1/len(sentence))
    perplexity = np.exp(perplexity)

    return perplexity


# Calculate PPL for interpolated trigram model
trigram_perplexity = []
for sentence in test_corpus:
    perplexity = calculate_trigram_perplexity(sentence, trigram_model, bigram_model, unigram_model, [0.1, 0.3, 0.6])
    trigram_perplexity.append(perplexity)

print(f"Interpolated Trigram Perplexity: {np.mean(trigram_perplexity)}")

Interpolated Trigram Perplexity: 13402.71588398677


Using your trained n-gram models to generate new texts with the greedy decoding startegy by completing the TODOs.

In [15]:
def generate_text_unigram(unigram_model, max_seq_len):
    """
    Generates text using the unigram model with greedy decoding.
    """
    current_token = "[BOS]"
    generated_text = [current_token]
    for _ in range(max_seq_len - 1):
        max_prob = 0
        for word in unigram_model:
            unigram_prob = unigram_model[word]

            # TODO: Select the next token using greedy decoding
            if unigram_prob>max_prob:
              max_prob=unigram_prob
              next_token = word

        generated_text.append(next_token)
        current_token = next_token

        if current_token == "[EOS]":
            break

    return generated_text

# Generate text using the unigram model
generated_text = generate_text_unigram(unigram_model, 10)
print(f"Unigram generated texts: {' '.join(generated_text)}")

Unigram generated texts: [BOS] the the the the the the the the the


In [16]:
def generate_text_bigram(bigram_model, unigram_model, max_seq_len):
    """
    Generates text using the bigram model with greedy decoding.
    """
    current_token = "[BOS]"
    generated_text = [current_token]
    for _ in range(max_seq_len - 1):
        max_prob = 0
        for word in unigram_model:
            bigram_prob = bigram_model.get((current_token, word), 0)

            # TODO: Select the next token using greedy decoding
            if bigram_prob > max_prob:
              max_prob = bigram_prob
              next_token = word

        generated_text.append(next_token)
        current_token = next_token

        if current_token == "[EOS]":
            break

    return generated_text

# Generate text using the bigram model
generated_text = generate_text_bigram(bigram_model, unigram_model, 10)
print(f"Bigram generated texts: {' '.join(generated_text)}")


Bigram generated texts: [BOS] the state college , the state college , the


In [22]:
def generate_text_trigram(trigram_model, bigram_model, unigram_model, max_seq_len):
    """
    Generates text using the trigram model with greedy decoding.
    """
    current_token_1 = "[BOS]"
    current_token_2 = None
    generated_text = [current_token_1]

    # Use bigram model to generate the second token
    max_prob = 0
    for word in unigram_model:
        # TODO: Select the second token using greedy decoding
        bigram_prob = bigram_model.get((current_token_1, word), 0)

        if bigram_prob > max_prob:
          max_prob = bigram_prob
          next_token = word

    generated_text.append(next_token)
    current_token_2 = next_token

    if current_token_2 == "[EOS]":
        return generated_text

    # Use trigram model to generate the remaining tokens

    for _ in range(max_seq_len - 2):
        max_prob = 0
        for word in unigram_model:

            # TODO: Select the next token using greedy decoding
          trigram_prob = trigram_model.get((current_token_1, current_token_2, word), 0)
          if trigram_prob > max_prob:
            max_prob = trigram_prob
            next_token = word

        generated_text.append(next_token)
        current_token_1 = current_token_2
        current_token_2 = next_token

        if current_token_2 == "[EOS]":
            break

    return generated_text

# Generate text using the trigram model
generated_text = generate_text_trigram(trigram_model, bigram_model, unigram_model, 15)
print(f"Trigram generated texts: {' '.join(generated_text)}")


Trigram generated texts: [BOS] the president said he was the first year in the early stages , garden
